In [1]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(loss)
    loss.backward()
    optimizer.step()

tensor(1.9476, grad_fn=<NllLossBackward>)
tensor(1.8307, grad_fn=<NllLossBackward>)
tensor(1.6957, grad_fn=<NllLossBackward>)
tensor(1.5587, grad_fn=<NllLossBackward>)
tensor(1.4129, grad_fn=<NllLossBackward>)
tensor(1.2781, grad_fn=<NllLossBackward>)
tensor(1.1640, grad_fn=<NllLossBackward>)
tensor(1.0467, grad_fn=<NllLossBackward>)
tensor(0.9149, grad_fn=<NllLossBackward>)
tensor(0.8570, grad_fn=<NllLossBackward>)
tensor(0.7226, grad_fn=<NllLossBackward>)
tensor(0.6447, grad_fn=<NllLossBackward>)
tensor(0.5797, grad_fn=<NllLossBackward>)
tensor(0.5493, grad_fn=<NllLossBackward>)
tensor(0.4433, grad_fn=<NllLossBackward>)
tensor(0.3867, grad_fn=<NllLossBackward>)
tensor(0.3447, grad_fn=<NllLossBackward>)
tensor(0.3564, grad_fn=<NllLossBackward>)
tensor(0.3166, grad_fn=<NllLossBackward>)
tensor(0.2670, grad_fn=<NllLossBackward>)
tensor(0.2276, grad_fn=<NllLossBackward>)
tensor(0.2513, grad_fn=<NllLossBackward>)
tensor(0.1837, grad_fn=<NllLossBackward>)
tensor(0.1837, grad_fn=<NllLossBac

In [6]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8040
